In [1]:
print("=== PROCEDURE, REFERENCE, & LIMITATIONS ===")
print("Procedure: Direct Design Method")
print("Reference: ACI 318M-14 | RCD by Darwin & Nelson")
print("Limitations: \n minimum 3x3 panel. If fewer, interior negative moments tend to be too small. \n Ratio not greater than 2. \n Span lengths not differ by more than 1/3 of the longer span. \n applicable only to braced frames \n Service live load shall not exceed two times the service dead load. \n relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.")

=== PROCEDURE, REFERENCE, & LIMITATIONS ===
Procedure: Direct Design Method
Reference: ACI 318M-14 | RCD by Darwin & Nelson
Limitations: 
 minimum 3x3 panel. If fewer, interior negative moments tend to be too small. 
 Ratio not greater than 2. 
 Span lengths not differ by more than 1/3 of the longer span. 
 applicable only to braced frames 
 Service live load shall not exceed two times the service dead load. 
 relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.


In [2]:
import math
pi = math.pi
def sqrt(x):
    return math.sqrt(x)
def rd(x):
    return round(x,2)

In [3]:
#Class and methods
class Beam:
    def __init__ (self, name, dimy, dimz, length):
        self.name = name
        self.dimy = dimy
        self.dimz = dimz
        self.length  = length
    def Icb(self):
        return self.dimy*self.dimz**3/12
class Slab:
    def __init__ (self, name, L, W, thk):
        self.name = name
        self.L = L #The larger dimension
        self.W = W 
        self.thk = thk
    def Ics(self):
        return self.W*self.thk**3/12

In [4]:
#Slab and Beam Material Properties
fc = 27 #MPa
Ec = rd(4700*sqrt(fc)/1000) #GPa, for normal weight concrete
fy = 230 #MPa
print("=== MATERIAL PROPERTIES ===")
print(f"fc = {fc}MPa | fy = {fy}MPa | Ec = {Ec}GPa")

#Beam Material and Geometric Properties
B_north = Beam("B_north",300,600,5400)
B_south = Beam("B_south",300,600,5400)
B_east = Beam("B_east",300,600,5400)
B_west = Beam("B_west",300,600,5400)

#Slab Material and Geometric Properties
print("=== GEOMETRIC PROPERTIES ===")
S1 = Slab("S1",5400,5400,100)
lb = S1.L #larger dimension
la = S1.W #shorter dimension
print(f"Initial Slab Geometric Properties\n L = {S1.L}mm, W = {S1.W}mm, thk = {S1.thk}mm.")
beta = S1.L / S1.W #Ratio of longer to shorter slab dim
print(f"Ratio of longer to shorter slab clear span: {rd(beta)}")

#Beam-to-slab Stiffness Ratio, alpha_f
alpha_f = B_north.Icb() / S1.Ics()
print(f"Average Relative Stiffness beam to slab, alpha_f = {rd(alpha_f)}.")


=== MATERIAL PROPERTIES ===
fc = 27MPa | fy = 230MPa | Ec = 24.42GPa
=== GEOMETRIC PROPERTIES ===
Initial Slab Geometric Properties
 L = 5400mm, W = 5400mm, thk = 100mm.
Ratio of longer to shorter slab clear span: 1.0
Average Relative Stiffness beam to slab, alpha_f = 12.0.


In [6]:
#Slab Minimum Thickness per 8.3.1.2 ACI318M-14
if alpha_f > 0.20 and alpha_f < 2:
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+5*(S1.L/S1.W)*(alpha_f - 0.2)) , 125)) #mm
else: 
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+9*(S1.L/S1.W)) , 90)) #mm
    
print(f"Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = {h_min}mm.")
#Factored Uniform Load
DL = 24*max(h_min,S1.thk)*.001 + 2.1 #selfweight of slab, kPa
LL = 2.4 #kPa
qu = 1.2*DL + 1.6*LL
print(f"Factored Uniform Pressure, qu = {rd(qu)}kPa.") #kPa

Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = 115.71mm.
Factored Uniform Pressure, qu = 9.69kPa.


In [10]:
#Minimum reinforcement near tension face for each dir
print("=== MIN. REINF. NEAR TENSION FACE, EA.DIR. ===")
print("*Per ACI318M-14, Section 8.6.1")
if fy < 420:
    Asmin = rd(0.0020 * 1000 * S1.thk)
    print(f"Asmin = 0.0020Ag = {Asmin}mm2 per mtr width of slab")
else:
    Asmin = rd( max( 0.0018*420*1000*S1.thk/fy , 0.0014*1000*S1.thk ) )
    print(f"Asmin = {Asmin}mm2 per mtr width of slab")

#Minimum spacing of flexural reinforcement at crit sections
print("Minimum spacing of flexural reinf. at crit. sections must not exceed 2*slab_thk per ACI318M-14, Section 8.7.3")

#Corner restraint in slab
print("=== CORNER RESTRAINT IN SLAB ===")
print("*Per ACI318M-14, Section 8.7.3")
lcor = int(rd(lb / 5)) #mm
print(f"Length corner bar, lcor = {lcor}mm")
Mu_cor = rd((1/2)*qu*(lcor*0.001)**2) #kN-m
print(f"Mu_cor = {Mu_cor}kN-m")


=== MIN. REINF. NEAR TENSION FACE, EA.DIR. ===
*Per ACI318M-14, Section 8.6.1
Asmin = 0.0020Ag = 200.0mm2 per mtr width of slab
Minimum spacing of flexural reinf. at crit. sections must not exceed 2*slab_thk per ACI318M-14, Section 8.7.3
=== CORNER RESTRAINT IN SLAB ===
*Per ACI318M-14, Section 8.7.3
Length corner bar, lcor = 1080mm
Mu_cor = 5.65kN-m


In [16]:
#Case: Simply Supported Square Slab, if only bending moments were present
print("=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===")
print("* Twisting moment relieves 25 percent of bending moment.")
print("* Bending moment formula: (qu/2)*l**2 / 8 less 25%")
Mu_square = ( ((qu*0.5)*(lb*0.001)**2) / 8 ) * 0.75 #kN-m
print(f"Mu_squareslab = {rd(Mu_square)}kN-m per mtr width")

=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===
* Twisting moment relieves 25 percent of bending moment.
* Bending moment formula: (qu/2)*l**2 / 8 less 25%
Mu_squareslab = 13.25kN-m per mtr width


In [19]:
#Total Statical Moment, Mo
Mo = qu*S1.W*.001*(S1.L*.001)**2 / 8 #kN-m, for each direction (N-S and E-W)
print(f"Total Statical Moment, Mo = {rd(Mo)} kN-m for each direction.")
#Widths of Middle and Column Strip...
width_cs = rd(min(0.5*S1.L,0.5*S1.W)) # 8.4.1.5
print(f"Width of Column Strip: {width_cs}mm") 
width_ms = rd(max(S1.L - width_cs, S1.W - width_cs))
print(f"Width of Middle Strip: {width_ms}mm")

Total Statical Moment, Mo = 190.78 kN-m for each direction.
Width of Column Strip: 2700.0mm
Width of Middle Strip: 2700.0mm


In [22]:
#Interior Panel Moments Distribution of Mo
print("Distribuition of Mo for Interior Panels")
Mo_neg = rd(0.65*Mo) #at supports/ends of slab
Mo_pos = rd(0.35*Mo) #at midspan of slab
print(f" Interior Panel Moment (negative), Mo_neg = {Mo_neg}kNm.")
print(f" Interior Panel Moment (positive), Mo_pos = {Mo_pos}kNm.")
#Interior Panel, Column Strip, Table 8.10.5.1
Mo_neg_cs = rd(0.65*Mo_neg)
print(f"  IPM_neg, Column Strip, Mo_neg_cs = {Mo_neg_cs}kNm.")
#Interior Panel, Middle Strip
Mo_neg_ms = rd(0.25*Mo_neg)
print(f"  IPM_neg, Middle Strip, Mo_neg_ms = {Mo_neg_ms} kNm")
#Pos Mu, Column Strip, Table 8.10.5.5
Mo_pos_cs = rd(0.75*Mo_pos)
print(f"  IPM_pos, Column Strip, Mo_pos_cs = {Mo_pos_cs} kNm")
Mo_pos_ms = rd(0.25*Mo_pos)
print(f"  IPM_pos, Mid Strip, Mo_pos_ms = {Mo_pos_ms} kNm")

Distribuition of Mo for Interior Panels
 Interior Panel Moment (negative), Mo_neg = 124.0kNm.
 Interior Panel Moment (positive), Mo_pos = 66.77kNm.
  IPM_neg, Column Strip, Mo_neg_cs = 80.6kNm.
  IPM_neg, Middle Strip, Mo_neg_ms = 31.0 kNm
  IPM_pos, Column Strip, Mo_pos_cs = 50.08 kNm
  IPM_pos, Mid Strip, Mo_pos_ms = 16.69 kNm


In [24]:
#End span Moments Distribution of Mo ACI318M-14 Table 8.10.4.2
print("Distribution of Mo for End Span with beams between all supports.")
Mo_eneg = rd(0.16*Mo)
Mo_ineg = rd(0.70*Mo)
Mo_epos = rd(0.57*Mo)
print(f" End Span Panel Moment (negative exterior), Mo_eneg = {Mo_eneg}kNm.")
print(f" End Span Panel Moment (negative interior), Mo_ineg = {Mo_ineg}kNm.")
print(f" End Span Panel Moment (positive), Mo_epos = {Mo_epos}kNm.")
Mo_eneg_cs = rd(0.65*Mo_eneg)
print(f"    ESPM_eneg, Col Strip, Mo_eneg_cs = {Mo_eneg_cs}kNm")
Mo_ineg_cs = rd(0.75*Mo_eneg)
print(f"    ESPM_ineg, Col Strip, Mo_ineg_cs = {Mo_ineg_cs}kNm")
Mo_epos_cs = rd(0.75*Mo_epos)
print(f"    ESPM_epos, Column Strip, Mo_epos_cs = {Mo_epos_cs} kNm")
Mo_eneg_ms = rd(Mo_eneg - Mo_eneg_cs)
print(f"    ESPM_eneg, Mid Strip, Mo_eneg_ms = {Mo_eneg_ms}kNm")
Mo_epos_ms = rd(Mo_epos - Mo_epos_cs)
print(f"    ESPM_epos, Mid Strip, Mo_epos_ms = {Mo_epos_ms} kNm")

Distribution of Mo for End Span with beams between all supports.
 End Span Panel Moment (negative exterior), Mo_eneg = 30.52kNm.
 End Span Panel Moment (negative interior), Mo_ineg = 133.54kNm.
 End Span Panel Moment (positive), Mo_epos = 108.74kNm.
    ESPM_eneg, Col Strip, Mo_eneg_cs = 19.84kNm
    ESPM_ineg, Col Strip, Mo_ineg_cs = 22.89kNm
    ESPM_epos, Column Strip, Mo_epos_cs = 81.55 kNm
    ESPM_eneg, Mid Strip, Mo_eneg_ms = 10.68kNm
    ESPM_epos, Mid Strip, Mo_epos_ms = 27.19 kNm
